In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from collections import Counter
import numpy as np
from numpy import loadtxt
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
p_scores = pd.read_csv('merfish_M1S1_filtered_periph_scores.csv')
p_scores = p_scores.loc[p_scores['annotation'] != 'unannotated']

In [3]:
# intensize computation: results stored in periphery_data.npy
num_unique_cells = (p_scores['cell_id']).unique().shape[0]
num_genes = (p_scores['gene']).unique().shape[0]
print("There are", num_genes, "genes")
print("There are", num_unique_cells, "unique cells")

annotation_indices = {k: v for v, k in enumerate(list(p_scores['annotation'].unique()))}
cell_indices = {k: v for v, k in enumerate(list(p_scores['cell_id'].unique()))}
gene_indices = {k: v for v, k in enumerate(list(p_scores['gene'].unique()))}

There are 192 genes
There are 17312 unique cells


In [4]:
# intensize computation: results stored in periphery_labels.npy
# x data shape(num_unique_cells, gene_periphery_scores)
data = np.zeros((num_unique_cells, num_genes))
for cell in cell_indices:
    subset = p_scores.loc[p_scores['cell_id'] == cell]
    for index, s in subset.iterrows():
        cell_index = cell_indices[cell]
        gene_index = gene_indices[s['gene']]
        data[cell_index][gene_index] = s['periphery_score']

KeyboardInterrupt: 

In [3]:
np.save('periphery_data', data)
np.save('periphery_labels', labels)

NameError: name 'data' is not defined

In [ ]:
# y data 
labels = np.zeros((num_unique_cells, len(annotation_indices)))
for cell in cell_indices:
    ann = list(p_scores.loc[p_scores['cell_id'] == cell]['annotation'])[0]
    arr_index = annotation_indices[ann]
    labels[cell_indices[cell]][arr_index] = 1

In [3]:
data = np.load('periphery_data.npy')
labels = np.load('periphery_labels.npy')
print(data.shape)
print(labels.shape)

(17312, 192)
(17312, 80)


In [11]:
# cv = np.std(data, axis=0) / np.mean(data, axis=0)
# data = data[:,np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99))]

In [20]:
expr = np.sum(np.square(data), axis=1)
d = data[np.logical_and(expr >= np.percentile(expr, 1), expr <= np.percentile(expr, 99)),]
l = labels[np.logical_and(expr >= np.percentile(expr, 1), expr <= np.percentile(expr, 99)),]
# cv = np.std(total_set, axis=1) / np.mean(total_set, axis=1)
# total_set = total_set[np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99)),]
# total_labels = total_labels[np.logical_and(cv >= np.percentile(cv, 1), cv <= np.percentile(cv, 99)),]

In [21]:
d.shape

(16971, 192)

In [22]:
l.shape

(16971, 80)

In [4]:
# split data into train, val and test datasets
train, test, train_y, test_y = train_test_split(data, labels, test_size=0.3)

test, val, test_y, val_y = train_test_split(test, test_y, test_size=0.5)
print(train.shape, train_y.shape)
print(val.shape, val_y.shape)
print(test.shape, test_y.shape)

(12118, 192) (12118, 80)
(2597, 192) (2597, 80)
(2597, 192) (2597, 80)


In [5]:
model = tf.keras.models.Sequential([tf.keras.layers.Input(192),
                                    tf.keras.layers.Dense(100, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(50, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(25, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(80, activation=tf.nn.softmax)])
lr= tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001,
    decay_steps=100000,
    decay_rate=0.95,
    staircase=True)
model.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])
model.summary()
model.fit(train, train_y, batch_size=128, epochs=300, validation_data=(val, val_y))

2021-11-05 23:09:00.824660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-05 23:09:00.921859: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-05 23:09:00.922699: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-05 23:09:00.924545: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               19300     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 80)                2080      
Total params: 27,705
Trainable params: 27,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
95/95 [==============================] - 2s 6ms/step - loss: 4.3638 - accuracy: 0.0458 - val_loss: 4.3433 - val_accuracy: 0.0593
Epoch 2/300
95/95 [==============================] - 0s 4ms/step - loss: 4.2942 - accuracy: 0.0680 - val_loss

95/95 [==============================] - 0s 4ms/step - loss: 2.2589 - accuracy: 0.3943 - val_loss: 2.4176 - val_accuracy: 0.3623
Epoch 53/300
95/95 [==============================] - 0s 4ms/step - loss: 2.2418 - accuracy: 0.3973 - val_loss: 2.4054 - val_accuracy: 0.3647
Epoch 54/300
95/95 [==============================] - 0s 4ms/step - loss: 2.2252 - accuracy: 0.4015 - val_loss: 2.3906 - val_accuracy: 0.3677
Epoch 55/300
95/95 [==============================] - 0s 4ms/step - loss: 2.2086 - accuracy: 0.4033 - val_loss: 2.3774 - val_accuracy: 0.3693
Epoch 56/300
95/95 [==============================] - 1s 6ms/step - loss: 2.1918 - accuracy: 0.4095 - val_loss: 2.3654 - val_accuracy: 0.3743
Epoch 57/300
95/95 [==============================] - 0s 4ms/step - loss: 2.1759 - accuracy: 0.4114 - val_loss: 2.3520 - val_accuracy: 0.3754
Epoch 58/300
95/95 [==============================] - 0s 4ms/step - loss: 2.1596 - accuracy: 0.4162 - val_loss: 2.3410 - val_accuracy: 0.3781
Epoch 59/300
95/95 

95/95 [==============================] - 1s 6ms/step - loss: 1.6154 - accuracy: 0.5470 - val_loss: 1.9674 - val_accuracy: 0.4790
Epoch 110/300
95/95 [==============================] - 1s 5ms/step - loss: 1.6085 - accuracy: 0.5477 - val_loss: 1.9644 - val_accuracy: 0.4829
Epoch 111/300
95/95 [==============================] - 1s 6ms/step - loss: 1.6015 - accuracy: 0.5501 - val_loss: 1.9614 - val_accuracy: 0.4829
Epoch 112/300
95/95 [==============================] - 1s 7ms/step - loss: 1.5947 - accuracy: 0.5501 - val_loss: 1.9573 - val_accuracy: 0.4813
Epoch 113/300
95/95 [==============================] - 0s 4ms/step - loss: 1.5881 - accuracy: 0.5531 - val_loss: 1.9550 - val_accuracy: 0.4844
Epoch 114/300
95/95 [==============================] - 0s 4ms/step - loss: 1.5817 - accuracy: 0.5557 - val_loss: 1.9506 - val_accuracy: 0.4836
Epoch 115/300
95/95 [==============================] - 0s 4ms/step - loss: 1.5750 - accuracy: 0.5567 - val_loss: 1.9487 - val_accuracy: 0.4871
Epoch 116/300

95/95 [==============================] - 0s 4ms/step - loss: 1.3109 - accuracy: 0.6224 - val_loss: 1.8584 - val_accuracy: 0.5148
Epoch 167/300
95/95 [==============================] - 0s 4ms/step - loss: 1.3063 - accuracy: 0.6243 - val_loss: 1.8576 - val_accuracy: 0.5156
Epoch 168/300
95/95 [==============================] - 0s 4ms/step - loss: 1.3026 - accuracy: 0.6250 - val_loss: 1.8582 - val_accuracy: 0.5191
Epoch 169/300
95/95 [==============================] - 0s 4ms/step - loss: 1.2984 - accuracy: 0.6269 - val_loss: 1.8552 - val_accuracy: 0.5171
Epoch 170/300
95/95 [==============================] - 0s 4ms/step - loss: 1.2947 - accuracy: 0.6263 - val_loss: 1.8544 - val_accuracy: 0.5179
Epoch 171/300
95/95 [==============================] - 0s 4ms/step - loss: 1.2902 - accuracy: 0.6290 - val_loss: 1.8560 - val_accuracy: 0.5187
Epoch 172/300
95/95 [==============================] - 0s 4ms/step - loss: 1.2861 - accuracy: 0.6300 - val_loss: 1.8544 - val_accuracy: 0.5171
Epoch 173/300

95/95 [==============================] - 0s 4ms/step - loss: 1.1108 - accuracy: 0.6762 - val_loss: 1.8595 - val_accuracy: 0.5279
Epoch 224/300
95/95 [==============================] - 0s 4ms/step - loss: 1.1079 - accuracy: 0.6771 - val_loss: 1.8592 - val_accuracy: 0.5295
Epoch 225/300
95/95 [==============================] - 0s 4ms/step - loss: 1.1050 - accuracy: 0.6785 - val_loss: 1.8632 - val_accuracy: 0.5241
Epoch 226/300
95/95 [==============================] - 0s 4ms/step - loss: 1.1019 - accuracy: 0.6792 - val_loss: 1.8624 - val_accuracy: 0.5264
Epoch 227/300
95/95 [==============================] - 0s 4ms/step - loss: 1.0988 - accuracy: 0.6811 - val_loss: 1.8638 - val_accuracy: 0.5283
Epoch 228/300
95/95 [==============================] - 0s 4ms/step - loss: 1.0959 - accuracy: 0.6816 - val_loss: 1.8626 - val_accuracy: 0.5264
Epoch 229/300
95/95 [==============================] - 0s 4ms/step - loss: 1.0934 - accuracy: 0.6801 - val_loss: 1.8638 - val_accuracy: 0.5287
Epoch 230/300

95/95 [==============================] - 0s 4ms/step - loss: 0.9632 - accuracy: 0.7209 - val_loss: 1.9216 - val_accuracy: 0.5337
Epoch 281/300
95/95 [==============================] - 0s 4ms/step - loss: 0.9613 - accuracy: 0.7211 - val_loss: 1.9199 - val_accuracy: 0.5329
Epoch 282/300
95/95 [==============================] - 0s 4ms/step - loss: 0.9591 - accuracy: 0.7235 - val_loss: 1.9244 - val_accuracy: 0.5333
Epoch 283/300
95/95 [==============================] - 0s 4ms/step - loss: 0.9571 - accuracy: 0.7231 - val_loss: 1.9293 - val_accuracy: 0.5352
Epoch 284/300
95/95 [==============================] - 0s 4ms/step - loss: 0.9541 - accuracy: 0.7239 - val_loss: 1.9258 - val_accuracy: 0.5372
Epoch 285/300
95/95 [==============================] - 0s 4ms/step - loss: 0.9523 - accuracy: 0.7245 - val_loss: 1.9251 - val_accuracy: 0.5356
Epoch 286/300
95/95 [==============================] - 0s 4ms/step - loss: 0.9499 - accuracy: 0.7257 - val_loss: 1.9280 - val_accuracy: 0.5360
Epoch 287/300

In [7]:
y_pred = model.predict(test)

In [8]:
y_pred.shape

(2597, 80)

In [12]:
np.where(y_pred

array([[6.9023841e-14, 2.1066073e-13, 1.1902239e-04, ..., 1.4169917e-04,
        8.0033584e-04, 1.0981495e-04],
       [1.4460075e-03, 3.9155670e-02, 1.2518999e-03, ..., 4.4073397e-03,
        4.5355261e-04, 6.9673844e-03],
       [2.9226914e-07, 1.7533808e-06, 1.5676622e-03, ..., 3.9922643e-02,
        2.7641291e-03, 1.8937675e-02],
       ...,
       [1.9470069e-14, 3.9199704e-01, 8.6634760e-10, ..., 2.5676108e-09,
        5.5166267e-14, 4.6950288e-10],
       [6.3872412e-03, 2.1324426e-02, 5.5265869e-04, ..., 2.6160762e-07,
        6.5091066e-09, 3.1180056e-07],
       [2.5853403e-22, 2.8989339e-31, 1.6093675e-10, ..., 1.0195894e-07,
        1.8279170e-06, 3.1377755e-07]], dtype=float32)

In [15]:
print(test_y.shape)
print(y_pred.shape)
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(test_y, axis=1)

(2597, 80)
(2597, 80)


In [16]:

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.1853624775845376